In [1]:
# TODO organize appropriately (consider specific ones below?)
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import seaborn as sns
#from IPython.core.debugger import Tracer
#import scipy as sp

import coorddescent

from pylab import rcParams
rcParams['figure.figsize'] = 5, 8

%matplotlib inline

# Exercise One

In [2]:
h = pd.read_csv('Hitters.csv', index_col=0)
h.shape

(322, 20)

In [3]:
h[:3]

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
Name,,,,,,,,,,,,,,,,,,,,
-Andy Allanson,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A


In [4]:
# remove rows w/ empty values - i think this is only those w/ no salary
h = h.dropna()
h.shape

(263, 20)

In [5]:
# convert categorical vars to dummy variables
h = pd.get_dummies(h)
h.shape

(263, 23)

While the homework doesn't state it specifically, I assume here that we're predicting salary, since that's what we've done previously.

In [6]:
# split into features and the value to predict (x and y)
# scale all features so each col has mean zero and std 1
X_scaled = preprocessing.scale(h.drop('Salary', axis=1))
Y = preprocessing.scale(h['Salary'])

(X_scaled.shape, Y.shape)

((263, 22), (263,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, random_state=0, test_size=0.25)

In [8]:
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((197, 22), (66, 22), (197,), (66,))

In [9]:
lambda_vals = [10**k for k in range(-5, 5)]
# increasing max_iter from default of 1000 to ensure convergence (per
# a warning that I'll get when it's the default), also setting 
# normalize to False as we've already standardized the data
lasso = LassoCV(alphas=lambda_vals, normalize=False, max_iter=3000)

lasso.fit(X_train, y_train)
print('Estimated lambda =', lasso.alpha_)
print('Estimated coefficients =', lasso.coef_)
print('Estimated intercept =', lasso.intercept_)

Estimated lambda = 0.0001
Estimated coefficients = [ -5.33261977e-01   5.65057051e-01   1.61724158e-01  -1.43237339e-01
  -3.34430414e-03   3.13334150e-01  -4.76383114e-02  -1.11350193e+00
   1.34670382e+00   2.42818751e-02   6.71324788e-01  -6.55211779e-02
  -4.00618231e-01   1.40596770e-01   1.16914735e-01  -8.33326487e-02
  -5.23084498e-02   0.00000000e+00   1.10185571e-01  -2.20934611e-14
   2.21390110e-02  -2.42440952e-16]
Estimated intercept = 0.0120197874567


In [13]:
foo = coorddescent.cycliccoorddescent(X_train, y_train, lasso.alpha_, max_iter=1000)
foo

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.371225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
1,0.371225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
2,0.371225,0.057800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
3,0.371225,0.057800,0.111543,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
4,0.371225,0.057800,0.111543,-0.057195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
5,0.371225,0.057800,0.111543,-0.057195,0.023470,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
6,0.371225,0.057800,0.111543,-0.057195,0.023470,0.097664,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
7,0.371225,0.057800,0.111543,-0.057195,0.023470,0.097664,0.328980,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
8,0.371225,0.057800,0.111543,-0.057195,0.023470,0.097664,0.328980,0.043189,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
9,0.371225,0.057800,0.111543,-0.057195,0.023470,0.097664,0.328980,0.043189,0.026187,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00


In [14]:
bar = coorddescent.randcoorddescent(X_train, y_train, lasso.alpha_, max_iter=1000)
bar

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014001,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014001,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014001,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014001,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.014551,-0.014001,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.014551,-0.014001,0.118737,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.014551,-0.014001,0.118737,0.000000,0.000000,0.000000
7,0.000000,0.290582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.014551,-0.014001,0.118737,0.000000,0.000000,0.000000
8,0.000000,0.290582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,-0.050943,-0.014551,-0.014001,0.118737,0.000000,0.000000,0.000000
9,0.000000,0.290582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000299,0.000000,...,0.000000,0.000000,0.000000,-0.050943,-0.014551,0.033093,0.118737,0.000000,0.000000,0.000000


In [ ]:
cd

In [ ]:
cd.lasso_objective()

In [ ]:
cd.c_term

In [ ]:
cd.